In [1]:
# Run in Jupyter (single cell) - updated base_dir
import pandas as pd
from pathlib import Path
import numpy as np

# ---------- change only if different ----------
base_dir = Path("/home/python/Downloads/Project3")
freq_path = base_dir / "freMTPL2freq.csv"
sev_path  = base_dir / "freMTPL2sev.csv"
# ----------------------------------------------

print("Files expected:", freq_path.exists(), sev_path.exists())
df_freq = pd.read_csv(freq_path)
df_sev  = pd.read_csv(sev_path)

# detect policy id
def find_col(df, candidates):
    cols_lower = {c.lower(): c for c in df.columns}
    for cand in candidates:
        if cand in df.columns:
            return cand
        if cand.lower() in cols_lower:
            return cols_lower[cand.lower()]
    return None

policy_candidates = ['IDpol','idpol','policy_id','policyid','policyID','pol_id']
freq_policy_col = find_col(df_freq, policy_candidates)
sev_policy_col  = find_col(df_sev, policy_candidates)

# detect metric cols
freq_candidates = ['ClaimNb','claimnb','claim_count','claim_nb','claims']
sev_candidates  = ['avg_severity','severity','ClaimAmount','claimamount','claim_amount','AvgClaimAmount','Severity']

freq_col = find_col(df_freq, freq_candidates)
sev_col  = find_col(df_sev, sev_candidates)

# rename and reduce
df_freq = df_freq.rename(columns={freq_policy_col: 'policy_id'})
df_sev  = df_sev.rename(columns={sev_policy_col:  'policy_id'})

df_freq_small = df_freq[['policy_id', freq_col]].copy() if freq_col else df_freq[['policy_id']].copy()
df_freq_small = df_freq_small.rename(columns={freq_col: 'frequency'})
df_sev_small  = df_sev[['policy_id', sev_col]].copy() if sev_col else df_sev[['policy_id']].copy()
df_sev_small  = df_sev_small.rename(columns={sev_col: 'severity'})

df_policy = pd.merge(df_freq_small, df_sev_small, on='policy_id', how='outer')
for col in ['frequency','severity']:
    df_policy[col] = pd.to_numeric(df_policy.get(col,0), errors='coerce').fillna(0)

# synthetic fraud flag (simple rule)
sev90 = df_policy['severity'].quantile(0.90)
freq90 = df_policy['frequency'].quantile(0.90)
df_policy['is_fraud'] = np.where((df_policy['severity'] > sev90) | (df_policy['frequency'] > freq90), 1, 0)

# save policy_metrics.csv
out_metrics = Path.home() / "policy_metrics.csv"
df_policy.to_csv(out_metrics, index=False)
print("Saved policy_metrics.csv to:", out_metrics)

# scoring (min-max -> 0..100 and weighted final score)
df = df_policy.copy()
metrics = ['severity', 'frequency', 'is_fraud']
for col in metrics:
    minv = df[col].min(); maxv = df[col].max()
    df[f'{col}_norm'] = 0.5 if (pd.isna(minv) or pd.isna(maxv) or maxv==minv) else (df[col]-minv)/(maxv-minv)

df['Severity_Score']  = df['severity_norm'] * 100
df['Frequency_Score'] = df['frequency_norm'] * 100
df['Fraud_Score']     = df['is_fraud_norm'] * 100
df['Final_Score'] = 0.4*df['Severity_Score'] + 0.3*df['Frequency_Score'] + 0.3*df['Fraud_Score']

low_cut = df['Final_Score'].quantile(0.33)
high_cut = df['Final_Score'].quantile(0.66)
df['Risk_Tier'] = df['Final_Score'].apply(lambda s: 'Low' if s<=low_cut else ('Medium' if s<=high_cut else 'High'))

out_final = Path.home() / "policy_risk_scores.csv"
df[['policy_id','frequency','severity','is_fraud','Severity_Score','Frequency_Score','Fraud_Score','Final_Score','Risk_Tier']].to_csv(out_final, index=False)
print("Saved policy_risk_scores.csv to:", out_final)
display(df.head())


Files expected: True True
Saved policy_metrics.csv to: /home/python/policy_metrics.csv
Saved policy_risk_scores.csv to: /home/python/policy_risk_scores.csv


,policy_id,frequency,severity,is_fraud,severity_norm,frequency_norm,is_fraud_norm,Severity_Score,Frequency_Score,Fraud_Score,Final_Score,Risk_Tier
0,1.0,1.0,0.0,1,0.0,0.0625,1.0,0.0,6.25,100.0,31.875,High
1,3.0,1.0,0.0,1,0.0,0.0625,1.0,0.0,6.25,100.0,31.875,High
2,5.0,1.0,0.0,1,0.0,0.0625,1.0,0.0,6.25,100.0,31.875,High
3,10.0,1.0,0.0,1,0.0,0.0625,1.0,0.0,6.25,100.0,31.875,High
4,11.0,1.0,0.0,1,0.0,0.0625,1.0,0.0,6.25,100.0,31.875,High


In [2]:
# Run in Jupyter
import pandas as pd
from sqlalchemy import create_engine
import getpass
from pathlib import Path

# ---------- CONFIG ----------
csv_path = Path.home() / "policy_metrics.csv"   # update if different
DB_USER = "root"        # change if needed
DB_PASS = "YOURPASSWORD"  # replace with your MySQL password or use input() below
DB_HOST = "localhost"
DB_PORT = 3306
DB_NAME = "insurance_db"
# --------------------------

# safer: comment previous line and use input() to avoid password in notebook:
# DB_PASS = getpass.getpass("MySQL password: ")

print("Loading CSV:", csv_path.exists(), csv_path)
df = pd.read_csv(csv_path)
print("Rows:", len(df))
display(df.head())

# Normalize column names expected
# Ensure policy_id string and numeric columns exist
df['policy_id'] = df['policy_id'].astype(str)
for col in ['frequency','severity','is_fraud']:
    if col not in df.columns:
        df[col] = 0
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)

# Connect to MySQL (create DB if not exists)
root_engine = create_engine(f"mysql+mysqlconnector://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/", echo=False)
with root_engine.connect() as conn:
    conn.execute(f"CREATE DATABASE IF NOT EXISTS {DB_NAME};")

engine = create_engine(f"mysql+mysqlconnector://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}", echo=False)

# Write table: replace if exists
df_to_write = df[['policy_id','frequency','severity','is_fraud']].copy()
df_to_write.to_sql('policy_metrics', engine, if_exists='replace', index=False, chunksize=1000, method='multi')

print("Uploaded to MySQL database:", DB_NAME, "table: policy_metrics")
# show top 5 rows from DB
print(pd.read_sql_query("SELECT * FROM policy_metrics LIMIT 5;", engine))


Loading CSV: True /home/python/policy_metrics.csv
Rows: 679708


,policy_id,frequency,severity,is_fraud
0,1.0,1.0,0.0,1
1,3.0,1.0,0.0,1
2,5.0,1.0,0.0,1
3,10.0,1.0,0.0,1
4,11.0,1.0,0.0,1


ProgrammingError: (mysql.connector.errors.ProgrammingError) 1698 (28000): Access denied for user 'root'@'localhost'
(Background on this error at: https://sqlalche.me/e/20/f405)

In [3]:
DB_USER = "proj_user"
DB_PASS = getpass.getpass("MySQL password for proj_user: ")
DB_NAME = "insurance_db"


MySQL password for proj_user:  ········


In [4]:
# Upload policy_metrics.csv to MySQL (runs in Jupyter)
import pandas as pd
from sqlalchemy import create_engine
import getpass
from pathlib import Path

csv_path = Path.home() / "policy_metrics.csv"   # should exist from earlier steps
print("CSV exists:", csv_path.exists(), csv_path)

# DB config
DB_USER = "proj_user"
DB_PASS = getpass.getpass("MySQL password for proj_user: ")
DB_HOST = "localhost"
DB_PORT = 3306
DB_NAME = "insurance_db"

# load CSV
df = pd.read_csv(csv_path)
print("Rows, cols:", df.shape)
display(df.head())

# minimal cleaning & typing
df['policy_id'] = df['policy_id'].astype(str)
for col in ['frequency','severity','is_fraud']:
    if col not in df.columns:
        df[col] = 0
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)

# create engine and write to MySQL
engine = create_engine(f"mysql+mysqlconnector://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}")
df[['policy_id','frequency','severity','is_fraud']].to_sql('policy_metrics', engine, if_exists='replace', index=False, chunksize=1000, method='multi')
print("Uploaded to MySQL: insurance_db.policy_metrics")

# quick check from DB
print(pd.read_sql_query("SELECT COUNT(*) AS total_rows FROM policy_metrics;", engine))
print(pd.read_sql_query("SELECT * FROM policy_metrics LIMIT 5;", engine))


CSV exists: True /home/python/policy_metrics.csv


MySQL password for proj_user:  ········


Rows, cols: (679708, 4)


,policy_id,frequency,severity,is_fraud
0,1.0,1.0,0.0,1
1,3.0,1.0,0.0,1
2,5.0,1.0,0.0,1
3,10.0,1.0,0.0,1
4,11.0,1.0,0.0,1


Uploaded to MySQL: insurance_db.policy_metrics
   total_rows
0      679708
  policy_id  frequency  severity  is_fraud
0       1.0        1.0       0.0         1
1       3.0        1.0       0.0         1
2       5.0        1.0       0.0         1
3      10.0        1.0       0.0         1
4      11.0        1.0       0.0         1


In [5]:
import pandas as pd
from pathlib import Path
p = Path.home() / "top100_high_risk.csv"
df = pd.read_csv(p)
print("Rows,Cols:", df.shape)
display(df.head(10))



Rows,Cols: (100, 5)


,policy_id,final_score,severity,frequency,is_fraud
0,1120377.0,0.701172,4.075401e+06,1,1
1,2241683.0,0.600218,2.220586e+03,256,1
2,3253234.0,0.442026,2.334947e+03,121,1
3,3254353.0,0.441954,1.598889e+03,121,1
4,2141337.0,0.428882,1.301173e+06,1,1
5,2248174.0,0.395122,2.039407e+03,81,1
6,3122016.0,0.377180,7.744115e+05,1,1
7,2239279.0,0.375305,3.103221e+03,64,1
8,110846.0,0.373598,7.020928e+05,4,1
9,2216294.0,0.342345,1.608933e+03,36,1
